## **Initialization and Parameter Setting**

The state of the five edges is represented by **component events**.  
Each edge can take a **binary state**:  
- `0`: Non-functional  
- `1`: Functional  

### **Edge Probabilities**
```python
probs = {
    'e1': {0: 0.01, 1: 0.99},
    'e2': {0: 0.01, 1: 0.99},
    'e3': {0: 0.05, 1: 0.95},
    'e4': {0: 0.05, 1: 0.95},
    'e5': {0: 0.10, 1: 0.90}
}

In [1]:
import itertools
import numpy as np
import networkx as nx
from scipy.stats import norm
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
from itertools import combinations

# Define the network
nodes = {'n1': (0, 0),
         'n2': (1, 1),
         'n3': (1, -1),
         'n4': (2, 0)}

edges = {'e1': ['n1', 'n2'],
         'e2': ['n1', 'n3'],
         'e3': ['n2', 'n3'],
         'e4': ['n2', 'n4'],
         'e5': ['n3', 'n4']}

probs = {'e1': 0.99, 'e2': 0.99, 'e3': 0.95, 'e4': 0.95, 'e5': 0.90}  # 엣지 신뢰도(survival probability)

## **Functions for Reliability Index and Redundancy Index**
- **Beta Calculation**
$$
\beta_{ij} = -\Phi^{-1}\left(P(F_i \mid H_j)\right)
$$

In [2]:
edge_names = list(probs.keys())
survival_probs = np.array([probs[edge] for edge in edge_names]) # 모든 엣지의 생존 확률

# 상태 조합 생성 (0: failure / 1: survival)
n_edges = len(edge_names)
state_combinations = list(itertools.product([0, 1], repeat=n_edges))

# Calculate failure probability
failure_probabilities = []
for state in state_combinations:
    probabilities = [
        survival_probs[i] if state[i] == 1 else (1 - survival_probs[i])
        for i in range(n_edges)
    ]
    failure_prob = np.prod(probabilities)
    failure_probabilities.append((state, failure_prob))

# Calculate beta
results = []
for state, failure_prob in failure_probabilities:
    if all(state):  # except [ 1, 1, 1, 1, 1 ] (all edges survive)
        continue
    if failure_prob > 0:  
        beta = -norm.ppf(failure_prob)
    else:
        beta = float('inf')  # 확률이 0일 경우 베타는 무한대
    results.append((state, failure_prob, beta))

# Print results
print(f"{'State':>20} | {'Failure Probability':>20} | {'Beta (β)':>10}")
print("-" * 60)
for state, failure_prob, beta in results:
    state_str = "".join(map(str, state))  # 상태를 문자열로 변환
    print(f"{state_str:>20} | {failure_prob:>20.10f} | {beta:>10.4f}")


               State |  Failure Probability |   Beta (β)
------------------------------------------------------------
               00000 |         0.0000000250 |     5.4513
               00001 |         0.0000002250 |     5.0465
               00010 |         0.0000004750 |     4.9017
               00011 |         0.0000042750 |     4.4509
               00100 |         0.0000004750 |     4.9017
               00101 |         0.0000042750 |     4.4509
               00110 |         0.0000090250 |     4.2877
               00111 |         0.0000812250 |     3.7712
               01000 |         0.0000024750 |     4.5669
               01001 |         0.0000222750 |     4.0825
               01010 |         0.0000470250 |     3.9055
               01011 |         0.0004232250 |     3.3371
               01100 |         0.0000470250 |     3.9055
               01101 |         0.0004232250 |     3.3371
               01110 |         0.0008934750 |     3.1235
               01111 |     

**MCS이용한 방법과 beta 결과 비교** - 거의 동일하게 나옴

모두 fail한 경우 [0,0,0,0,0]의 베타값이 차이가 좀 크게 나옴
이유 : MCS의 경우 케이스 별 확률 값에 비례하게 sampling되는데 해당 케이크스의 경우 Pf=0.0000000250로, 한 번도 샘플링되지 않음 -> 튀어보이는 결과 도출

In [3]:
# 첫 번째 결과 (Failure Probability and Beta values)
result1 = [
    ("00000", 5.4513), ("00001", 5.0465), ("00010", 4.9017), ("00011", 4.4509),
    ("00100", 4.9017), ("00101", 4.4509), ("00110", 4.2877), ("00111", 3.7712),
    ("01000", 4.5669), ("01001", 4.0825), ("01010", 3.9055), ("01011", 3.3371),
    ("01100", 3.9055), ("01101", 3.3371), ("01110", 3.1235), ("01111", 2.4070),
    ("10000", 4.5669), ("10001", 4.0825), ("10010", 3.9055), ("10011", 3.3371),
    ("10100", 3.9055), ("10101", 3.3371), ("10110", 3.1235), ("10111", 2.4070),
    ("11000", 3.4861), ("11001", 2.8472), ("11010", 2.6004), ("11011", 1.7291),
    ("11100", 2.6004), ("11101", 1.7291), ("11110", 1.3503)
]

# 두 번째 결과 (Scenario Subset and Beta values, flipped states)
result2 = [
    ("11110", 1.349946), ("11101", 1.728809), ("11100", 2.600920), ("11011", 1.730381),
    ("11010", 2.603534), ("11001", 2.845886), ("11000", 3.493015), ("10111", 2.406913),
    ("10110", 3.123062), ("10101", 3.335194), ("10100", 3.908164), ("10011", 3.326957),
    ("10010", 3.907127), ("10001", 4.067109), ("10000", 4.506196), ("01111", 2.407004),
    ("01110", 3.127800), ("01101", 3.334412), ("01100", 3.906610), ("01011", 3.340658),
    ("01010", 3.888177), ("01001", 4.082259), ("01000", 4.573344), ("00111", 3.763428),
    ("00110", 4.331644), ("00101", 4.476153), ("00100", 4.935367), ("00011", 4.454727),
    ("00010", 4.753424), ("00001", 5.199338), ("00000", 7.034484)
]

# Convert results to dictionaries for easy comparison
beta1_dict = dict(result1)
beta2_dict = dict(result2)

# Compare beta values for matching states
comparison = []
for state in beta1_dict.keys():
    if state in beta2_dict:
        beta1 = beta1_dict[state]
        beta2 = beta2_dict[state]
        difference = abs(beta1 - beta2)
        comparison.append((state, beta1, beta2, difference))

# Sort by difference for better analysis
comparison = sorted(comparison, key=lambda x: x[3], reverse=True)

# Output results
print(f"{'State':>10} | {'Beta1':>10} | {'Beta2':>10} | {'Difference':>10}")
print("-" * 50)
for state, beta1, beta2, diff in comparison:
    print(f"{state:>10} | {beta1:>10.4f} | {beta2:>10.4f} | {diff:>10.4f}")


     State |      Beta1 |      Beta2 | Difference
--------------------------------------------------
     00000 |     5.4513 |     7.0345 |     1.5832
     00001 |     5.0465 |     5.1993 |     0.1528
     00010 |     4.9017 |     4.7534 |     0.1483
     10000 |     4.5669 |     4.5062 |     0.0607
     00110 |     4.2877 |     4.3316 |     0.0439
     00100 |     4.9017 |     4.9354 |     0.0337
     00101 |     4.4509 |     4.4762 |     0.0253
     01010 |     3.9055 |     3.8882 |     0.0173
     10001 |     4.0825 |     4.0671 |     0.0154
     10011 |     3.3371 |     3.3270 |     0.0101
     00111 |     3.7712 |     3.7634 |     0.0078
     11000 |     3.4861 |     3.4930 |     0.0069
     01000 |     4.5669 |     4.5733 |     0.0064
     01110 |     3.1235 |     3.1278 |     0.0043
     00011 |     4.4509 |     4.4547 |     0.0038
     01011 |     3.3371 |     3.3407 |     0.0036
     11010 |     2.6004 |     2.6035 |     0.0031
     01101 |     3.3371 |     3.3344 |     0.0027

- **Pi Calculation**

$$
\begin{align*}
\pi_{ij} & = -\Phi^{-1}\left(P(F_{\text{sys}} \mid F_i, H_j)\right) \\
         & = -\Phi^{-1}\left(\frac{P(F_{\text{sys}}, F_i)}{P(F_i)}\right) \\
         & = -\Phi^{-1}\left(
         \frac{
         \sum_{b = (\boldsymbol{l}, \boldsymbol{u}, p) \in B_f : \boldsymbol{u}\langle X_i \rangle = 0} p
         + \sum_{b = (\boldsymbol{l}, \boldsymbol{u}, p) \in B_f : \boldsymbol{u}\langle X_i \rangle = 1 \text{ and } \boldsymbol{l}\langle X_i \rangle = 0} \left(p \cdot P(F_i)\right)
         + \sum_{b = (\boldsymbol{l}, \boldsymbol{u}, p) \in B_f : \boldsymbol{u}\langle X_i \rangle = 1 \text{ and } \boldsymbol{l}\langle X_i \rangle = 1} 0
         }{P(F_i)}
         \right)
\end{align*}
$$